<a href="https://colab.research.google.com/github/pratzz/Customer-Churn-Prediction/blob/main/Predict_Customer_Churn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
 
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
import pandas as pd
from pandas_profiling import ProfileReport

In [3]:
train_data = pd.read_csv("/gdrive/MyDrive/dataset/CustomerChurn/train.csv")
test_data = pd.read_csv("/gdrive/MyDrive/dataset/CustomerChurn/test.csv")

In [4]:
train_data.head()

,customer_id,Name,age,gender,security_no,region_category,membership_category,joining_date,joined_through_referral,referral_id,preferred_offer_types,medium_of_operation,internet_option,last_visit_time,days_since_last_login,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback,churn_risk_score
0,fffe4300490044003600300030003800,Pattie Morrisey,18,F,XW0DQ7H,Village,Platinum Membership,2017-08-17,No,xxxxxxxx,Gift Vouchers/Coupons,?,Wi-Fi,16:08:02,17,300.63,53005.25,17.0,781.75,Yes,Yes,No,Not Applicable,Products always in Stock,2
1,fffe43004900440032003100300035003700,Traci Peery,32,F,5K0N3X1,City,Premium Membership,2017-08-28,?,CID21329,Gift Vouchers/Coupons,Desktop,Mobile_Data,12:38:13,16,306.34,12838.38,10.0,NaN,Yes,No,Yes,Solved,Quality Customer Care,1
2,fffe4300490044003100390032003600,Merideth Mcmeen,44,F,1F2TCL3,Town,No Membership,2016-11-11,Yes,CID12313,Gift Vouchers/Coupons,Desktop,Wi-Fi,22:53:21,14,516.16,21027.00,22.0,500.69,No,Yes,Yes,Solved in Follow-up,Poor Website,5
3,fffe43004900440036003000330031003600,Eufemia Cardwell,37,M,VJGJ33N,City,No Membership,2016-10-29,Yes,CID3793,Gift Vouchers/Coupons,Desktop,Mobile_Data,15:57:50,11,53.27,25239.56,6.0,567.66,No,Yes,Yes,Unsolved,Poor Website,5
4,fffe43004900440031003900350030003600,Meghan Kosak,31,F,SVZXCWB,City,No Membership,2017-09-12,No,xxxxxxxx,Credit/Debit Card Offers,Smartphone,Mobile_Data,15:46:44,20,113.13,24483.66,16.0,663.06,No,Yes,Yes,Solved,Poor Website,5


In [5]:
print("Train shape:",train_data.shape)
print("Test shape:",test_data.shape)

Train shape: (36992, 25)
Test shape: (19919, 24)


In [6]:
# !pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

In [7]:
# ProfileReport(train_data, title="Pandas Profiling Report")

In [8]:
# ProfileReport(test_data, title="Pandas Profiling Report")

In [9]:
train_data['data']='train'
test_data['data']='test'
data = pd.concat([train_data,test_data],axis=0, sort=True)
data.head()

,Name,age,avg_frequency_login_days,avg_time_spent,avg_transaction_value,churn_risk_score,complaint_status,customer_id,data,days_since_last_login,feedback,gender,internet_option,joined_through_referral,joining_date,last_visit_time,medium_of_operation,membership_category,offer_application_preference,past_complaint,points_in_wallet,preferred_offer_types,referral_id,region_category,security_no,used_special_discount
0,Pattie Morrisey,18,17.0,300.63,53005.25,2.0,Not Applicable,fffe4300490044003600300030003800,train,17,Products always in Stock,F,Wi-Fi,No,2017-08-17,16:08:02,?,Platinum Membership,Yes,No,781.75,Gift Vouchers/Coupons,xxxxxxxx,Village,XW0DQ7H,Yes
1,Traci Peery,32,10.0,306.34,12838.38,1.0,Solved,fffe43004900440032003100300035003700,train,16,Quality Customer Care,F,Mobile_Data,?,2017-08-28,12:38:13,Desktop,Premium Membership,No,Yes,NaN,Gift Vouchers/Coupons,CID21329,City,5K0N3X1,Yes
2,Merideth Mcmeen,44,22.0,516.16,21027.00,5.0,Solved in Follow-up,fffe4300490044003100390032003600,train,14,Poor Website,F,Wi-Fi,Yes,2016-11-11,22:53:21,Desktop,No Membership,Yes,Yes,500.69,Gift Vouchers/Coupons,CID12313,Town,1F2TCL3,No
3,Eufemia Cardwell,37,6.0,53.27,25239.56,5.0,Unsolved,fffe43004900440036003000330031003600,train,11,Poor Website,M,Mobile_Data,Yes,2016-10-29,15:57:50,Desktop,No Membership,Yes,Yes,567.66,Gift Vouchers/Coupons,CID3793,City,VJGJ33N,No
4,Meghan Kosak,31,16.0,113.13,24483.66,5.0,Solved,fffe43004900440031003900350030003600,train,20,Poor Website,F,Mobile_Data,No,2017-09-12,15:46:44,Smartphone,No Membership,Yes,Yes,663.06,Credit/Debit Card Offers,xxxxxxxx,City,SVZXCWB,No


# Drop customer id, name, security no, last visit time









In [10]:
test_id = pd.DataFrame(data = data[data['data'] == 'test']['customer_id'])
data.drop(["customer_id", "Name", "security_no"],axis=1,inplace=True)

*   Joined through referral and medium of operation has ? indicating missing 
*   avg frq login days has a category error






In [11]:
import numpy as np

data = data.replace({'?': np.nan, 'Error': np.nan})

# Handling missing values

In [12]:
data.isna().sum()

age                                 0
avg_frequency_login_days         5437
avg_time_spent                      0
avg_transaction_value               0
churn_risk_score                19919
complaint_status                    0
data                                0
days_since_last_login               0
feedback                            0
gender                              0
internet_option                     0
joined_through_referral          8436
joining_date                        0
last_visit_time                     0
medium_of_operation              8357
membership_category                 0
offer_application_preference        0
past_complaint                      0
points_in_wallet                 5406
preferred_offer_types             447
referral_id                         0
region_category                  8376
used_special_discount               0
dtype: int64

In [13]:
data['joined_through_referral'].fillna(train_data['joined_through_referral'].mode()[0], inplace=True)
data['medium_of_operation'].fillna(train_data['medium_of_operation'].mode()[0], inplace=True)
data['points_in_wallet'].fillna(train_data['points_in_wallet'].mean(), inplace=True)
data['preferred_offer_types'].fillna(train_data['preferred_offer_types'].mode()[0], inplace=True)
data['region_category'].fillna(train_data['region_category'].mode()[0], inplace=True)

train_data["avg_frequency_login_days"] = pd.to_numeric(train_data["avg_frequency_login_days"].replace({'Error': np.nan})).abs().round()
data["avg_frequency_login_days"] = pd.to_numeric(data["avg_frequency_login_days"]).abs().round()
data['avg_frequency_login_days'].fillna(train_data["avg_frequency_login_days"].median(), inplace=True)

In [14]:
data.isna().sum()

age                                 0
avg_frequency_login_days            0
avg_time_spent                      0
avg_transaction_value               0
churn_risk_score                19919
complaint_status                    0
data                                0
days_since_last_login               0
feedback                            0
gender                              0
internet_option                     0
joined_through_referral             0
joining_date                        0
last_visit_time                     0
medium_of_operation                 0
membership_category                 0
offer_application_preference        0
past_complaint                      0
points_in_wallet                    0
preferred_offer_types               0
referral_id                         0
region_category                     0
used_special_discount               0
dtype: int64

# Joining date needs to be separated in day, month, year


In [15]:
data["joining_date"] = pd.to_datetime(data["joining_date"])

data['join_year'] = data['joining_date'].dt.year 
data['join_month'] = data['joining_date'].dt.month 
data['join_day'] = data['joining_date'].dt.day

data.drop(["joining_date"],axis=1,inplace=True)

# Handling negative values

Days since last login, avg time spent, points in wallet, churn risk score has negative values

In [16]:
# Since all negative values are -999, we will fill them with mean of positive values
days_since_last_login_mean = round(train_data[train_data['days_since_last_login']>0]['days_since_last_login'].mean())
data[data['days_since_last_login']<0]['days_since_last_login'] = days_since_last_login_mean

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [17]:
data['avg_time_spent'] = data['avg_time_spent'].abs()

In [18]:
data['churn_risk_score'] = data['churn_risk_score'].abs()

Dropping irrelevant columns

In [19]:
data.drop(["last_visit_time", "referral_id"],axis=1,inplace=True)

# Boolean columns

In [20]:
data['joined_through_referral'] = data['joined_through_referral'].replace({'Yes': 1, 'No': 0})
data['offer_application_preference'] = data['offer_application_preference'].replace({'Yes': 1, 'No': 0})
data['past_complaint'] = data['past_complaint'].replace({'Yes': 1, 'No': 0})
data['used_special_discount'] = data['used_special_discount'].replace({'Yes': 1, 'No': 0})

# Separate train and test data

In [21]:
train_data = data[data['data']=='train']
del train_data['data']

test_data = data[data['data']=='test']
del test_data['data']
del test_data['churn_risk_score']

In [22]:
print("Data shape:",data.shape)
print("Train shape:",train_data.shape)
print("Test shape:",test_data.shape)

Data shape: (56911, 23)
Train shape: (36992, 22)
Test shape: (19919, 21)


In [23]:
# ProfileReport(train_data, title="Pandas Profiling Report")

# Convert Categorical columns

In [24]:
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict

d = defaultdict(LabelEncoder)
cat_cols = ["complaint_status","feedback","gender","internet_option",
            "medium_of_operation","membership_category",
            "preferred_offer_types","region_category"]
train_data.loc[:, cat_cols] = train_data[cat_cols].apply(lambda x: d[x.name].fit_transform(x))

test_data.loc[:, cat_cols] = test_data[cat_cols].apply(lambda x: d[x.name].transform(x))

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1715: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, v)


# Scaling the input

In [25]:
# from sklearn.preprocessing import MinMaxScaler

# train_data = train_data.reset_index()
# test_data = test_data.reset_index()
# cols = ["avg_time_spent", "avg_transaction_value", "points_in_wallet"]
# scaler = MinMaxScaler()
# train_data[cols] = pd.DataFrame(scaler.fit_transform(train_data[cols]), columns=cols)
# test_data[cols] = pd.DataFrame(scaler.transform(test_data[cols]), columns=cols)

# Splitting data

In [26]:
from sklearn.model_selection import train_test_split

train_data_x = train_data.loc[:, train_data.columns != 'churn_risk_score']
train_data_y = train_data["churn_risk_score"]
X_train, X_valid, y_train, y_valid = train_test_split(train_data_x, train_data_y, test_size=0.3, random_state=0)

In [27]:
# from sklearn.linear_model import LogisticRegression
# from sklearn import metrics

# logreg = LogisticRegression()
# logreg.fit(X_train, y_train)

In [28]:
# importing random forest classifier from assemble module 
from sklearn.ensemble import RandomForestClassifier 

# creating a RF classifier 
clf = RandomForestClassifier()   
  
# Training the model on the training dataset 
# fit function is used to train the model using the training sets as parameters 
clf.fit(X_train, y_train) 

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [29]:
from sklearn.model_selection import GridSearchCV

# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [110, 130, 150],
    'max_features': [5, 10, 15, 20],
    'min_samples_leaf': [4],
    'min_samples_split': [12, 15, 20],
    'n_estimators': [500, 1000]
}
# Create a based model
rf = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [30]:
# Fit the grid search to the data
grid_search.fit(X_train, y_train)
grid_search.best_params_

Fitting 3 folds for each of 72 candidates, totalling 216 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed: 16.3min
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed: 98.1min
[Parallel(n_jobs=-1)]: Done 216 out of 216 | elapsed: 140.7min finished


{'bootstrap': True,
 'max_depth': 110,
 'max_features': 15,
 'min_samples_leaf': 4,
 'min_samples_split': 20,
 'n_estimators': 1000}

In [27]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

In [ ]:
best_grid = grid_search.best_estimator_

evaluate(best_grid, X_train, y_train)
evaluate(best_grid, X_valid, y_valid)

In [37]:
feature_imp = pd.Series(best_grid.feature_importances_,index=X_train.columns).sort_values(ascending=False)
feature_imp

points_in_wallet                0.390037
membership_category             0.244749
feedback                        0.163811
avg_transaction_value           0.060920
avg_time_spent                  0.025099
avg_frequency_login_days        0.019161
age                             0.018336
join_day                        0.016529
days_since_last_login           0.015836
join_month                      0.011338
complaint_status                0.005378
internet_option                 0.004153
preferred_offer_types           0.004100
join_year                       0.004008
region_category                 0.003841
medium_of_operation             0.003216
gender                          0.002439
joined_through_referral         0.002170
past_complaint                  0.001675
offer_application_preference    0.001626
used_special_discount           0.001578
dtype: float64

In [ ]:
!pip install tpot

In [29]:
from tpot import TPOTClassifier
GENERATIONS = 10
POP_SIZE = 100
CV = 3

tpot = TPOTClassifier(
    generations=GENERATIONS,
    population_size=POP_SIZE,
    random_state=123,
    n_jobs=-1,
    # use_dask=True,
    periodic_checkpoint_folder="/gdrive/MyDrive/dataset/CustomerChurn/",
    cv=CV,
    verbosity=2,
)

tpot.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)



Generation 1 - Current best internal CV score: 0.7617982155621766

Generation 2 - Current best internal CV score: 0.7617982155621766

Generation 3 - Current best internal CV score: 0.7617982155621766

Generation 4 - Current best internal CV score: 0.7617982155621766

Generation 5 - Current best internal CV score: 0.7617982155621766

Generation 6 - Current best internal CV score: 0.7617982155621766

Generation 7 - Current best internal CV score: 0.7617982155621766

Generation 8 - Current best internal CV score: 0.7617982155621766

Generation 9 - Current best internal CV score: 0.7651966967000825

Generation 10 - Current best internal CV score: 0.7651966967000825

Best pipeline: DecisionTreeClassifier(MinMaxScaler(DecisionTreeClassifier(input_matrix, criterion=gini, max_depth=7, min_samples_leaf=15, min_samples_split=14)), criterion=entropy, max_depth=7, min_samples_leaf=5, min_samples_split=9)


TPOTClassifier(config_dict=None, crossover_rate=0.1, cv=3,
               disable_update_check=False, early_stop=None, generations=10,
               log_file=None, max_eval_time_mins=5, max_time_mins=None,
               memory=None, mutation_rate=0.9, n_jobs=-1, offspring_size=None,
               periodic_checkpoint_folder='/gdrive/MyDrive/dataset/CustomerChurn/',
               population_size=100, random_state=123, scoring=None,
               subsample=1.0, template=None, use_dask=False, verbosity=2,
               warm_start=False)

In [30]:
evaluate(tpot, X_train, y_train)
evaluate(tpot, X_valid, y_valid)

Model Performance
Average Error: 0.2754 degrees.
Accuracy = 86.20%.
Model Performance
Average Error: 0.2971 degrees.
Accuracy = 84.92%.


84.92340962335552

In [31]:
tpot.export('/gdrive/MyDrive/dataset/CustomerChurn/tpot_carnival_pipeline.py')

In [32]:
y_test_pred=tpot.predict(test_data)

test_id["churn_risk_score"] = y_test_pred

In [33]:
test_id.to_csv('./submission.csv')

In [34]:
tpot.evaluated_individuals_

{'RandomForestClassifier(input_matrix, RandomForestClassifier__bootstrap=True, RandomForestClassifier__criterion=gini, RandomForestClassifier__max_features=0.35000000000000003, RandomForestClassifier__min_samples_leaf=18, RandomForestClassifier__min_samples_split=12, RandomForestClassifier__n_estimators=100)': {'crossover_count': 0,
  'generation': 0,
  'internal_cv_score': 0.7505215598721007,
  'mutation_count': 0,
  'operator_count': 1,
  'predecessor': ('ROOT',)},
 'DecisionTreeClassifier(input_matrix, DecisionTreeClassifier__criterion=entropy, DecisionTreeClassifier__max_depth=7, DecisionTreeClassifier__min_samples_leaf=4, DecisionTreeClassifier__min_samples_split=5)': {'crossover_count': 0,
  'generation': 0,
  'internal_cv_score': 0.7598673125341845,
  'mutation_count': 0,
  'operator_count': 1,
  'predecessor': ('ROOT',)},
 'MLPClassifier(ExtraTreesClassifier(input_matrix, ExtraTreesClassifier__bootstrap=False, ExtraTreesClassifier__criterion=entropy, ExtraTreesClassifier__max_f

In [35]:
import pickle

with open('./random_forest_grid_searched_2.pickle', 'wb') as f:
    pickle.dump(best_grid, f)

In [85]:
with open('./random_forest_grid_searched.pickle', 'rb') as f:
    rf = pickle.load(f)